In [1]:
from manim import *

Manim Community v0.18.0

In [2]:
import random
import os

In [3]:
#config.media_width = "75%"
config.verbosity = "WARNING"
config.background_color = "#24043d"

In [ ]:
%%manim -ql Mazo
class Mazo(Scene):
    def construct(self):
        as_pica = ImageMobject("..\\images\\1p.png").scale(2)
        self.add(as_pica)
        self.wait(2)
    

In [4]:
class Mazo:
    """La clase utiliza una ruta donde estarán las cartas y un parámetro con valor por defecto para la escala """
    def __init__(self, ruta_carpeta_cartas, escala=1.5):
        self.ruta = ruta_carpeta_cartas
        self.escala = escala
        self.cartas = self._cargar_cartas()
        self.barajar()

    def _cargar_cartas(self):
        """Carga todas las imágenes .png de la carpeta y las convierte en ImageMobjects."""
        cartas = []
        for archivo in sorted(os.listdir(self.ruta)):
            if archivo.endswith(".png"):
                carta = ImageMobject(os.path.join(self.ruta, archivo)).scale(self.escala)
                cartas.append(carta)
        return cartas

    def barajar(self):
        random.shuffle(self.cartas)

    def tomar(self, cantidad=1):
        """Toma las primeras 'cantidad' cartas del mazo."""
        seleccion = self.cartas[:cantidad]
        self.cartas = self.cartas[cantidad:]
        return seleccion

    def resetear(self):
        """Recarga y baraja el mazo (útil para repetir la animación)."""
        self.cartas = self._cargar_cartas()
        self.barajar()

    def mostrar_en_fila(self, start=LEFT * 4, espacio=1):
        """Devuelve un Group con las cartas en fila."""            # Las imágenes no pueden formar un VGroup, eso es solo para figuras vectorizadas.
        fila = Group(*self.cartas).arrange(RIGHT, buff=espacio).move_to(start)    
        return fila

    def apilar_cartas(self, direccion=DOWN, desplazamiento=0.3):
        """
        Devuelve un Group donde las cartas que se apilan parcialmente.    
        dirección: vector de desplazamiento entre cartas (ej. DOWN, RIGHT).
        desplazamiento: cuánto se desplaza cada carta respecto a la anterior.
        """
        pila = Group()
        for i, carta in enumerate(self.cartas):
            carta_copy = carta.copy().shift(i * desplazamiento * direccion)
            pila.add(carta_copy)
        return pila

    def animar_apilamiento(self, escena, n=None, direccion=DOWN, desplazamiento=0.4, duracion=0.5):
        """
        Anima una pila de cartas con desplazamiento parcial.
        
        - escena: instancia de Scene donde se reproducirá.
        - n: cantidad de cartas a mostrar (por defecto, todas).
        - direccion: vector de dirección del apilamiento.
        - desplazamiento: cuánto se ve de la carta de abajo.
        - duracion: duración de la animación por carta.
        """
        if n is None:
            n = len(self.cartas)

        seleccionadas = self.cartas[:n]

        # Recorremos al revés para que la última carta quede arriba, esto afecta al vector dirección, entonces DOWN desplazará las cartas hacia
        # arriba en la animación y viceversa
        for i, carta in enumerate(reversed(seleccionadas)):
            idx = n - 1 - i  # índice desde el final hacia el principio
            carta.move_to(ORIGIN + idx * desplazamiento * direccion)
            escena.add(carta)
            escena.play(FadeIn(carta), run_time=duracion)


In [ ]:
%%manim -ql MostrarCartas
class MostrarCartas(Scene):
    def construct(self):
        mazo = Mazo("../images/cartas", escala=1.5)

        fila = mazo.mostrar_en_fila(start=ORIGIN, espacio=0.5)
        self.play(FadeIn(fila))
        self.wait(2)

        cartas_repartidas = mazo.tomar(3)
        for carta in cartas_repartidas:
            self.play(carta.animate.shift(UP * 2), run_time=0.5)
        self.wait()


In [ ]:
%%manim -ql ApilarCartas
class ApilarCartas(Scene):
    def construct(self):
        mazo = Mazo("../images/cartas", escala=1.5)
        cartas = mazo.cartas  # Accedemos a las cartas directamente

        # Punto base
        base = ORIGIN
        direccion = DOWN
        desplazamiento = 0.25
        duracion = 0.8  # 🕒 tiempo entre cada carta

        for i, carta in enumerate(cartas):
            carta.move_to(base + i * desplazamiento * direccion)
            self.add(carta)  # Agrega la carta pero sin mostrarla aún
            self.play(FadeIn(carta), run_time=duracion)

        self.wait()

In [9]:
%%manim -qh ApilarArbitrario
class ApilarArbitrario(Scene):
    def construct(self):
        mazo = Mazo("../images/cartas", escala=1.5)
        # n es la cantidad de cartas que voy a apilar
        mazo.animar_apilamiento(self, n=3, direccion=UP, desplazamiento=0.3, duracion=0.8)
        self.wait()